# Converter

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from converter import convert_func
import scipy.stats as stats
from glob import glob
import os
from os.path import join
DATA_FOLDER = "./data/"
CAM_NAME = "camera3"

# DATA_FOLDER = "./data2/"
# CAM_NAME = "camera2"

# DATA_FOLDER = "./data0/"
# CAM_NAME = "camera0"

# DATA_FOLDER = "./data1/"
# CAM_NAME = "camera1"



"""
1. Generate _cs_nt_Pixel_eDep.csv using Geant4
2. Save noise data as npy in data_xu_R using parameter_estimation.ipynb
3. Generate noisy movie using gamma_noise_image_convert.ipynb 
4. Estimate leakage points using leakage_estimation.ipynb
"""

if False:
    DATA_FOLDER = "./data_xu_L/"
    video_fname = "./turbine_testv5_L_0010-0082.mp4"
else:
    DATA_FOLDER = "./data_xu_R/"
    video_fname = "./turbine_testv5_R_0010-0082.mp4"

CAM_NAME = "camera"

%matplotlib inline

In [ ]:
print(DATA_FOLDER)
print(video_fname)

# Load groundtruth noise

In [ ]:
cap = cv2.VideoCapture(video_fname)
cap.isOpened()

In [ ]:
wo_noise_imgs = []
while True:
    ret, img = cap.read()
    if not ret:
        break
    wo_noise_imgs.append(img)

In [ ]:
len(wo_noise_imgs)

In [ ]:
for it in wo_noise_imgs:
    cv2.imshow('img', it)
    key = cv2.waitKey(300)
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
plt.imshow(wo_noise_imgs[0][:,:,::-1])

# Load optim functio

In [ ]:
# Load optim function
# json_fname = f"convert_params_{CAM_NAME}.json"
json_fname = f"convert_params_camera3.json"
with open(json_fname) as f:
    params = json.load(f)
    
original_fps = 12
video_fps = 24
print(f"origial fps: {original_fps}")
print(f"video fps: {video_fps}")
print("Adjust max_val")
params["max_val"] = params["max_val"] * original_fps / video_fps 
print(params)

# Generate noise image

In [ ]:
geant4_deposits = []
for i in range(1000):
    try:
        deposit = np.load(f"{DATA_FOLDER}/geant4_gamma_noise_{i}.npy")
        geant4_deposits.append(deposit)
    except:
        print("Finish loading: Total", i)
        break

In [ ]:
from converter import normalize_image, jpeg_block_noise
def convert_func_color(geant4_deposit, wo_noise, **kwargs):
    #     print('kwargs: ', kwargs)
    geant4_noise = geant4_deposit.copy()
    geant4_noise = cv2.GaussianBlur(geant4_noise, (3, 3), kwargs["blur_sigma"])
    geant4_noise_wo_jpeg = normalize_image(
        geant4_noise, min_val=0, max_val=kwargs["max_val"]
    )
    geant4_noise = jpeg_block_noise(
        geant4_noise_wo_jpeg, quality=kwargs["jpeg_quality"]
    )

    if geant4_noise.dtype != np.uint8:
        geant4_noise = (255 * geant4_noise).astype(np.uint8)

    # absolute noise model
    noisy_img = wo_noise.copy()
    mask = np.any(geant4_noise[...,None] > wo_noise, axis=-1)
    noisy_img[mask] = geant4_noise[mask][..., None]

    return noisy_img

In [ ]:
len(wo_noise_imgs)

In [ ]:
# Generate gamma noise image
noisy_imgs = []
for deposit, wo_noise in zip(geant4_deposits, wo_noise_imgs):
    noisy_img = convert_func_color(deposit, wo_noise, **params)
    noisy_imgs.append(noisy_img)

In [ ]:
for it in noisy_imgs:
    cv2.imshow('img', it)
    key = cv2.waitKey(100)
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
h,w,c = noisy_imgs[0].shape
frame_rate = 24.0
size = (w, h)
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') 
writer = cv2.VideoWriter(video_fname.replace(".mp4", "_noisy.mp4"), fmt, frame_rate, size)
for it in noisy_imgs:
    writer.write(it)
writer.release()